
<h1>ENCS5343 Computer Vision</h1>




<h2>1. Introduction</h2>
        <p>This assignment investigates the performance of various noise reduction filters in image processing. By applying different noise types to images and then removing it through filtering, we aim to compare each filter's effectiveness in terms of Mean Squared Error (MSE), Peak Signal-to-Noise Ratio (PSNR), edge preservation, and computational efficiency. This analysis will provide insights into the most suitable filters for real-world applications, considering their trade-offs between noise removal, edge retention, and processing speed.</p>

<p>The objective is to compare each filter's effectiveness and efficiency, evaluating Mean Squared Error (MSE), Peak Signal-to-Noise Ratio (PSNR), edge preservation, and computational time across different kernel sizes. </p>
 

## Table of Contents

1. [Step 1: Generate or Load Noisy Images](#step-1-generate-or-load-noisy-images)
   - 1.1 [Upload Clean Images](#upload-clean-images)
   - 1.2 [Add Noise](#add-noise)
2. [Step 2: Apply Filters](#step-2-apply-filters)
   - 2.1 [Simple Filters](#simple-filters)
     - 4.1.1 [Box Filter](#box-filter)
       - 2.1.1.1 [Gaussian Noise](#gaussian-noise)
       - 2.1.1.2 [Salt and Pepper Noise](#salt-and-pepper-noise)
     - 2.1.2 [Gaussian Filter](#gaussian-filter)
       - 2.1.2.1 [Gaussian Noise](#gaussian-noise-1)
       - 2.1.2.2 [Salt and Pepper Noise](#salt-and-pepper-noise-1)
     - 2.1.3 [Median Filter](#median-filter)
       - 2.1.3.1 [Gaussian Noise](#gaussian-noise-2)
       - 2.1.3.2 [Salt and Pepper Noise](#salt-and-pepper-noise-2)
   - 2.2 [Advanced Filters](#advanced-filters)
     - 2.2.1 [Adaptive Median Filter](#adaptive-median-filter)
       - 2.2.1.1 [Gaussian Noise](#gaussian-noise-3)
       - 2.2.1.2 [Salt and Pepper Noise](#salt-and-pepper-noise-3)
     - 2.2.2 [Bilateral Filter](#bilateral-filter)
       - 2.2.2.1 [Gaussian Noise](#gaussian-noise-4)
       - 2.2.2.2 [Salt and Pepper Noise](#salt-and-pepper-noise-4)
     - 2.2.3 [Adaptive Mean Filter](#adaptive-mean-filter)
       - 2.2.3.1 [Gaussian Noise](#gaussian-noise-5)
       - 2.2.3.2 [Salt and Pepper Noise](#salt-and-pepper-noise-5)
3. [Step 3: Measure Performance](#step-3-measure-performance)
   - 3.1 [Mean Squared Error (MSE)](#mse)
   - 3.2 [Peak Signal-to-Noise Ratio (PSNR)](#psnr)
   - 3.3 [Edge Preservation](#edge-preservation)
3. [Conclusion](#conclusion)


https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from utils import *
from Calculation import *
from AdaptiveFilters import *

# Step 1: Generate or load noisy images

### upload clean images

<p>Four images with varying levels of edge complexity and different internal details were selected to evaluate the performance of different noise reduction filters. These images will help in understanding how each filter handles noise while preserving edges and details.</p>



In [ ]:
images = load_images_from_folder('../images')

In [ ]:
#show images as grid 2 in each row
fig, axs = plt.subplots(len(images)//2, 2, figsize=(10, 10))
for i, img in enumerate(images):
    axs[i//2, i%2].imshow(img, cmap='gray')
    axs[i//2, i%2].axis('off')
plt.show()



In [ ]:
for img in images:
    print(f"Image shape: {img.shape}")

### Add Noise

<ul>
            <li><strong>Gaussian Noise</strong>: Random noise with a normal distribution, simulating sensor or lighting irregularities.</li>
            <li><strong>Salt-and-Pepper Noise</strong>: Sporadic pixel intensities of maximum or minimum values, mimicking image transmission errors.</li>
        </ul>

In [ ]:
# {0:{gaussian: [], salt_pepper: []}, 1:{gaussian: [], salt_pepper: []}, ...}
images_with_noise = {i: {'gaussian': [], 'salt_pepper': []} for i in range(len(images))}

gaus_values = [20, 50, 100]
salt_pepper_values = [0.05, 0.2, 0.5]

for i, img in enumerate(images):

    for value in gaus_values:
        images_with_noise[i]['gaussian'].append(add_gaussian_noise(img, sigma=value))

    for value in salt_pepper_values:
        images_with_noise[i]['salt_pepper'].append(add_salt_pepper_noise(img, amount=value))

In [ ]:

for i in range(len(images)):
    print(f"Showing Gaussian noise for image {i+1}:")
    fig, axs = plt.subplots(1, len(gaus_values)+1, figsize=(10, 10))
    axs[0].imshow(images[i], cmap='gray')
    axs[0].axis('off')
    axs[0].set_title('Original')

    for j, img in enumerate(images_with_noise[i]['gaussian']):
        axs[j+1].imshow(img, cmap='gray')
        axs[j+1].axis('off')
        axs[j+1].set_title(f'Gaussian noise: {gaus_values[j]}')

    plt.show()


In [ ]:

for i in range(len(images)):
    print(f"Showing salt_pepper noise for image {i+1}:")
    fig, axs = plt.subplots(1, len(salt_pepper_values)+1, figsize=(10, 10))
    axs[0].imshow(images[i], cmap='gray')
    axs[0].axis('off')
    axs[0].set_title('Original')

    for j, img in enumerate(images_with_noise[i]['salt_pepper']):
        axs[j+1].imshow(img, cmap='gray')
        axs[j+1].axis('off')
        axs[j+1].set_title(f'salt_pepper noise: {salt_pepper_values[j]}')

    plt.show()


# Step 2: Apply filters



<ol>
            <li><strong>Simple Filters</strong>: Evaluates basic filters such as Box, Gaussian, and Median filters for noise reduction.</li>
            <li><strong>Advanced Filters</strong>: Explores combinations and custom implementations to enhance noise reduction across various noise types.</li>
    </ol>

In [ ]:
kernel_sizes_to_show = [3,15, 35]

kernel_sizes = range(3, 36, 4)
for k in kernel_sizes:
    print(f"Kernel size: {k}")
   

## Simple Filters

<ol>
            <li><strong>Box Filter</strong>: An averaging filter that smoothens images by replacing each pixel’s value with the mean of its neighbors.</li>
            <li><strong>Gaussian Filter</strong>: A filter that applies a Gaussian function to assign more weight to the central pixels in the neighborhood, which effectively reduces Gaussian noise while preserving some detail.</li>
            <li><strong>Median Filter</strong>: Designed specifically to handle salt-and-pepper noise, it replaces each pixel’s value with the median of its neighborhood, retaining edges better than the box filter.</li>
            
</ol>

### Box Filter

#### Gaussian Noise

In [ ]:

#images_with_box_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

images_with_box_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

time_images_with_box_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in kernel_sizes:
            start_time = time.time()
            images_with_box_filter_gaussian[i][j].append(cv2.blur(images_with_noise[i]['gaussian'][j], (k, k)))
            time_images_with_box_filter_gaussian[i][j].append((time.time() - start_time)*1000)


plot_with_noise_filtered(images=images,
                          images_with_noise=images_with_noise,
                            images_with_filter=images_with_box_filter_gaussian,
                              noise_type='gaussian',
                                filter_type='box',
                                  noise_values=gaus_values,
                                    kernel_sizes=kernel_sizes,
                                      kernel_sizes_to_show=kernel_sizes_to_show)


#### Salt_Pepper Noise

In [ ]:

#images_with_box_filter_salt_peper = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

images_with_box_filter_salt_pepper = {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}
time_images_with_box_filter_salt_pepper = {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}


for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in kernel_sizes:
            start_time = time.time()
            images_with_box_filter_salt_pepper[i][j].append(cv2.blur(images_with_noise[i]['salt_pepper'][j], (k, k)))
            time_images_with_box_filter_salt_pepper[i][j].append((time.time() - start_time)*1000)


plot_with_noise_filtered(images=images,
                          images_with_noise=images_with_noise,
                            images_with_filter=images_with_box_filter_salt_pepper,
                              noise_type='salt_pepper',
                                filter_type='box',
                                  noise_values=salt_pepper_values,
                                    kernel_sizes=kernel_sizes,
                                      kernel_sizes_to_show=kernel_sizes_to_show)

### Gaussian Filter

#### Gaussian Noise

In [ ]:

#images_with_gaussian_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

images_with_gaussian_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

time_images_with_gaussian_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in kernel_sizes:
            start_time = time.time()
            images_with_gaussian_filter_gaussian[i][j].append(cv2.GaussianBlur(images_with_noise[i]['gaussian'][j], (k, k),0))
            time_images_with_gaussian_filter_gaussian[i][j].append((time.time() - start_time)*1000)


plot_with_noise_filtered(images=images,
                            images_with_noise=images_with_noise,
                                images_with_filter=images_with_gaussian_filter_gaussian,
                                noise_type='gaussian',
                                    filter_type='gaussian',
                                    noise_values=gaus_values,
                                        kernel_sizes=kernel_sizes,
                                        kernel_sizes_to_show=kernel_sizes_to_show)

#### Salt_Pepper Noise

In [ ]:

#images_with_gaussian_filter_salt_pepper = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

images_with_gaussian_filter_salt_pepper = {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}

time_images_with_gaussian_filter_salt_pepper = {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in kernel_sizes:
            start_time = time.time()
            images_with_gaussian_filter_salt_pepper[i][j].append(cv2.GaussianBlur(images_with_noise[i]['salt_pepper'][j], (k, k),0))

            time_images_with_gaussian_filter_salt_pepper[i][j].append((time.time() - start_time)*1000)


plot_with_noise_filtered(images=images,
                            images_with_noise=images_with_noise,
                                images_with_filter=images_with_gaussian_filter_salt_pepper,
                                noise_type='salt_pepper',
                                    filter_type='gaussian',
                                    noise_values=salt_pepper_values,
                                        kernel_sizes=kernel_sizes,
                                        kernel_sizes_to_show=kernel_sizes_to_show)

### Median Filter

#### Gaussian Noise

In [ ]:

#images_with_median_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

images_with_median_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

time_images_with_median_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in kernel_sizes:
            # here kernel as k
            start_time = time.time()
            images_with_median_filter_gaussian[i][j].append(cv2.medianBlur(images_with_noise[i]['gaussian'][j],  k))
            time_images_with_median_filter_gaussian[i][j].append((time.time() - start_time)*1000)


plot_with_noise_filtered(images=images,
                            images_with_noise=images_with_noise,
                                images_with_filter=images_with_median_filter_gaussian,
                                noise_type='gaussian',
                                    filter_type='median',
                                    noise_values=gaus_values,
                                        kernel_sizes=kernel_sizes,
                                        kernel_sizes_to_show=kernel_sizes_to_show)


#### Salt_Pepper Noise

In [ ]:

#images_with_median_filter_salt_pepper = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

images_with_median_filter_salt_pepper = {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}

time_images_with_median_filter_salt_pepper = {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in kernel_sizes:
            # here kernel as k
            start_time = time.time()
            images_with_median_filter_salt_pepper[i][j].append(cv2.medianBlur(images_with_noise[i]['salt_pepper'][j],  k))
            time_images_with_median_filter_salt_pepper[i][j].append((time.time() - start_time)*1000)


plot_with_noise_filtered(images=images,
                            images_with_noise=images_with_noise,
                                images_with_filter=images_with_median_filter_salt_pepper,
                                noise_type='salt_pepper',
                                    filter_type='median',
                                    noise_values=salt_pepper_values,
                                        kernel_sizes=kernel_sizes,
                                        kernel_sizes_to_show=kernel_sizes_to_show)

## Advanced filters

### Adaptive Median Filter

https://www.irjet.net/archives/V6/i10/IRJET-V6I10148.pdf

#### Gaussian Noise

In [ ]:

#images_with_box_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

images_with_AdaptiveMedian_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}
time_images_with_AdaptiveMedian_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}
# time_images_with_box_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in kernel_sizes:
            start_time = time.time()
            # images_with_box_filter_gaussian[i][j].append(cv2.blur(images_with_noise[i]['gaussian'][j], (k, k)))
            images_with_AdaptiveMedian_filter_gaussian[i][j].append(adaptiveMedianFilter(images_with_noise[i]['gaussian'][j], max_kernel_size=k))
            time_images_with_AdaptiveMedian_filter_gaussian[i][j].append((time.time() - start_time)*1000)
            # images_with_AdaptiveMedian_filter_gaussian[i][j].append((time.time() - start_time)*1000)


plot_with_noise_filtered(images=images,
                            images_with_noise=images_with_noise,
                                images_with_filter=images_with_AdaptiveMedian_filter_gaussian,
                                noise_type='gaussian',
                                    filter_type='AdaptiveMedian',
                                    noise_values=gaus_values,
                                        kernel_sizes=kernel_sizes,
                                        kernel_sizes_to_show=kernel_sizes_to_show)

#### Salt_Pepper Noise

In [ ]:

#images_with_box_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

images_with_AdaptiveMedian_filter_salt_pepper = {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}
time_images_with_AdaptiveMedian_filter_salt_pepper = {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}
# time_images_with_box_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in kernel_sizes:
            start_time = time.time()
            # images_with_box_filter_gaussian[i][j].append(cv2.blur(images_with_noise[i]['gaussian'][j], (k, k)))
            images_with_AdaptiveMedian_filter_salt_pepper[i][j].append(adaptiveMedianFilter(images_with_noise[i]['salt_pepper'][j], max_kernel_size=k))
            time_images_with_AdaptiveMedian_filter_salt_pepper[i][j].append((time.time() - start_time)*1000)
            # images_with_AdaptiveMedian_filter_gaussian[i][j].append((time.time() - start_time)*1000)


plot_with_noise_filtered(images=images,
                            images_with_noise=images_with_noise,
                                images_with_filter=images_with_AdaptiveMedian_filter_salt_pepper,
                                noise_type='salt_pepper',
                                    filter_type='AdaptiveMedian',
                                    noise_values=salt_pepper_values,
                                        kernel_sizes=kernel_sizes,
                                        kernel_sizes_to_show=kernel_sizes_to_show)

### Bilateral Filter

#### Gaussian Noise

In [ ]:

#images_with_median_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

images_with_Bilateral_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

time_images_with_Bilateral_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in kernel_sizes:
            # here kernel as k
            start_time = time.time()
            images_with_Bilateral_filter_gaussian[i][j].append(cv2.bilateralFilter(images_with_noise[i]['gaussian'][j],  k,75 , 75 ))
            # bilateralFilter(src, d (can be as kernel size?), sigmaColor, sigmaSpace)

            time_images_with_Bilateral_filter_gaussian[i][j].append((time.time() - start_time)*1000)


plot_with_noise_filtered(images=images,
                            images_with_noise=images_with_noise,
                                images_with_filter=images_with_Bilateral_filter_gaussian,
                                noise_type='gaussian',
                                    filter_type='Bilateral',
                                    noise_values=gaus_values,
                                        kernel_sizes=kernel_sizes,
                                        kernel_sizes_to_show=kernel_sizes_to_show)

#### Salt_Pepper Noise

In [ ]:

#images_with_median_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

images_with_Bilateral_filter_salt_pepper = {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}
time_images_with_Bilateral_filter_salt_pepper = {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}


for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in kernel_sizes:
            # here kernel as k
            start_time = time.time()
            images_with_Bilateral_filter_salt_pepper[i][j].append(cv2.bilateralFilter(images_with_noise[i]['salt_pepper'][j],  k,75 , 75 ))
            # bilateralFilter(src, d (can be as kernel size?), sigmaColor, sigmaSpace)
            time_images_with_Bilateral_filter_salt_pepper[i][j].append((time.time() - start_time)*1000)


plot_with_noise_filtered(images=images,
                            images_with_noise=images_with_noise,
                                images_with_filter=images_with_Bilateral_filter_salt_pepper,
                                noise_type='salt_pepper',
                                    filter_type='Bilateral',
                                    noise_values=salt_pepper_values,
                                        kernel_sizes=kernel_sizes,
                                        kernel_sizes_to_show=kernel_sizes_to_show)

### Adaptive Mean Filter

#### Gaussian Noise

In [ ]:

#images_with_box_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

images_with_AdaptiveMean_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}
time_images_with_AdaptiveMean_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}
# time_images_with_box_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in kernel_sizes:
            start_time = time.time()
            # images_with_box_filter_gaussian[i][j].append(cv2.blur(images_with_noise[i]['gaussian'][j], (k, k)))
            images_with_AdaptiveMean_filter_gaussian[i][j].append(adaptiveMeanFilter(images_with_noise[i]['gaussian'][j], max_kernel_size=k))
            time_images_with_AdaptiveMean_filter_gaussian[i][j].append((time.time() - start_time)*1000)
            # images_with_AdaptiveMedian_filter_gaussian[i][j].append((time.time() - start_time)*1000)

plot_with_noise_filtered(images=images,
                            images_with_noise=images_with_noise,
                                images_with_filter=images_with_AdaptiveMean_filter_gaussian,
                                noise_type='gaussian',
                                    filter_type='AdaptiveMean',
                                    noise_values=gaus_values,
                                        kernel_sizes=kernel_sizes,
                                        kernel_sizes_to_show=kernel_sizes_to_show)

#### Salt_Pepper Noise

In [ ]:

#images_with_box_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

images_with_AdaptiveMean_filter_salt_pepper = {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}
time_images_with_AdaptiveMean_filter_salt_pepper = {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}
# time_images_with_box_filter_gaussian = {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in kernel_sizes:
            start_time = time.time()
            # images_with_box_filter_gaussian[i][j].append(cv2.blur(images_with_noise[i]['gaussian'][j], (k, k)))
            images_with_AdaptiveMean_filter_salt_pepper[i][j].append(adaptiveMedianFilter(images_with_noise[i]['salt_pepper'][j], max_kernel_size=k))
            time_images_with_AdaptiveMean_filter_salt_pepper[i][j].append((time.time() - start_time)*1000)
            # images_with_AdaptiveMedian_filter_gaussian[i][j].append((time.time() - start_time)*1000)

plot_with_noise_filtered(images=images,
                            images_with_noise=images_with_noise,
                                images_with_filter=images_with_AdaptiveMean_filter_salt_pepper,
                                noise_type='salt_pepper',
                                    filter_type='AdaptiveMean',
                                    noise_values=salt_pepper_values,
                                        kernel_sizes=kernel_sizes,
                                        kernel_sizes_to_show=kernel_sizes_to_show)

# Step 3: Measure performance

## Simple Filters

### Box Filter

#### MSE

In [ ]:


MSE_images_with_box_filter_gaussian = calculate_MSE_for_filters(images, images_with_box_filter_gaussian, kernel_sizes)
MSE_images_with_box_filter_salt_pepper = calculate_MSE_for_filters(images, images_with_box_filter_salt_pepper, kernel_sizes)

MSE_images_with_gaussian_filter_gaussian= calculate_MSE_for_filters(images, images_with_gaussian_filter_gaussian, kernel_sizes)
MSE_images_with_gaussian_filter_salt_pepper= calculate_MSE_for_filters(images, images_with_gaussian_filter_salt_pepper, kernel_sizes)


MSE_images_with_median_filter_gaussian= calculate_MSE_for_filters(images, images_with_median_filter_gaussian, kernel_sizes)
MSE_images_with_median_filter_salt_pepper= calculate_MSE_for_filters(images, images_with_median_filter_salt_pepper, kernel_sizes)

MSE_images_with_AdaptiveMean_filter_gaussian= calculate_MSE_for_filters(images, images_with_AdaptiveMean_filter_gaussian, kernel_sizes)
MSE_images_with_AdaptiveMean_filter_salt_pepper= calculate_MSE_for_filters(images, images_with_AdaptiveMean_filter_salt_pepper, kernel_sizes)

MSE_images_with_Bilateral_filter_gaussian= calculate_MSE_for_filters(images, images_with_Bilateral_filter_gaussian, kernel_sizes)
MSE_images_with_Bilateral_filter_salt_pepper= calculate_MSE_for_filters(images, images_with_Bilateral_filter_salt_pepper, kernel_sizes)


MSE_images_with_AdaptiveMedian_filter_gaussian= calculate_MSE_for_filters(images, images_with_AdaptiveMedian_filter_gaussian, kernel_sizes)
MSE_images_with_AdaptiveMedian_filter_salt_pepper= calculate_MSE_for_filters(images, images_with_AdaptiveMedian_filter_salt_pepper, kernel_sizes)


PSNR_images_with_box_filter_gaussian= calculate_PSNR_for_filters(images, images_with_box_filter_gaussian, kernel_sizes)
PSNR_images_with_box_filter_salt_pepper= calculate_PSNR_for_filters(images, images_with_box_filter_salt_pepper, kernel_sizes)


PSNR_images_with_gaussian_filter_gaussian= calculate_PSNR_for_filters(images, images_with_gaussian_filter_gaussian, kernel_sizes)
PSNR_images_with_gaussian_filter_salt_pepper= calculate_PSNR_for_filters(images, images_with_gaussian_filter_salt_pepper, kernel_sizes)


PSNR_images_with_median_filter_gaussian= calculate_PSNR_for_filters(images, images_with_median_filter_gaussian, kernel_sizes)
PSNR_images_with_median_filter_salt_pepper= calculate_PSNR_for_filters(images, images_with_median_filter_salt_pepper, kernel_sizes)


PSNR_images_with_AdaptiveMean_filter_gaussian= calculate_PSNR_for_filters(images, images_with_AdaptiveMean_filter_gaussian, kernel_sizes)
PSNR_images_with_AdaptiveMean_filter_salt_pepper= calculate_PSNR_for_filters(images, images_with_AdaptiveMean_filter_salt_pepper, kernel_sizes)


PSNR_images_with_Bilateral_filter_gaussian= calculate_PSNR_for_filters(images, images_with_Bilateral_filter_gaussian, kernel_sizes)
PSNR_images_with_Bilateral_filter_salt_pepper= calculate_PSNR_for_filters(images, images_with_Bilateral_filter_salt_pepper, kernel_sizes)

PSNR_images_with_AdaptiveMedian_filter_gaussian= calculate_PSNR_for_filters(images, images_with_AdaptiveMedian_filter_gaussian, kernel_sizes)
PSNR_images_with_AdaptiveMedian_filter_salt_pepper= calculate_PSNR_for_filters(images, images_with_AdaptiveMedian_filter_salt_pepper, kernel_sizes)


#### Edge preservation

In [ ]:
original_edges = [cv2.Canny(img, 100, 200) for img in images]

##### Gaussian Noise

In [ ]:
EDGES_images_with_box_filter_gaussian= {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in range(len(kernel_sizes)) :
            # print(i, j, k)
            EDGES_images_with_box_filter_gaussian[i][j].append(cv2.Canny(images_with_box_filter_gaussian[i][j][k], 100, 200))

In [ ]:



# for i in range(len(images)):
#     print_with_border(f"Showing Edges for image {i + 1} with Box filter for Gaussian noise:")

#     for j in range(len(gaus_values)):
#         print(f"\n\tShowing Edges for Gaussian noise {gaus_values[j]}:")

#         fig, axs = plt.subplots(1, len(kernel_sizes)+1, figsize=(10, 10))

#         axs[0].imshow(original_edges[i], cmap='gray')
#         axs[0].axis('off')
#         axs[0].set_title('Original Edges')

#         for k in range(len(kernel_sizes)):
#             axs[k+1].imshow(EDGES_images_with_box_filter_gaussian[i][j][k], cmap='gray')
#             axs[k+1].axis('off')
#             axs[k+1].set_title(f'Edges: Box filter: {kernel_sizes[k]}')

#         plt.tight_layout()

#         plt.show()

##### Salt_Pepper Noise

In [ ]:

EDGES_images_with_box_filter_salt_pepper= {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in range(len(kernel_sizes)) :
            # print(i, j, k)
            EDGES_images_with_box_filter_salt_pepper[i][j].append(cv2.Canny(images_with_box_filter_salt_pepper[i][j][k], 100, 200))

            
                                           


In [ ]:

for i in range(len(images)):
    print_with_border(f"Showing Edges for image {i + 1} with Box filter for Salt_Pepper noise:")

    for j in range(len(salt_pepper_values)):
        print(f"\n\tShowing Edges for Salt_Pepper noise {salt_pepper_values[j]}:")

        fig, axs = plt.subplots(1, len(kernel_sizes)+1, figsize=(10, 10))

        axs[0].imshow(original_edges[i], cmap='gray')
        axs[0].axis('off')
        axs[0].set_title('Original Edges')

        for k in range(len(kernel_sizes)):
            axs[k+1].imshow(EDGES_images_with_box_filter_salt_pepper[i][j][k], cmap='gray')
            axs[k+1].axis('off')
            axs[k+1].set_title(f'Edges: Box filter: {kernel_sizes[k]}')

        plt.tight_layout()

        plt.show()

### Gaussian Filter

#### MSE

In [ ]:
#images_with_gaussian_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}





In [ ]:
# plot each image on same plot plot for each gaus value
for i in range(len(images)):
    # print_with_border(f"Showing MSE for image {i + 1}:")
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for j in range(len(gaus_values)):
        axs[0].plot(kernel_sizes, MSE_images_with_gaussian_filter_gaussian[i][j], label=f'Gaussian noise: {gaus_values[j]}')
       
    for k in range(len(salt_pepper_values)):
        axs[1].plot(kernel_sizes, MSE_images_with_gaussian_filter_salt_pepper[i][k], label=f'Salt_Pepper noise: {salt_pepper_values[k]}')
    
    
    axs[0].set_xlabel('Kernel size')
    axs[0].set_ylabel('MSE')
    axs[0].set_title(f'MSE for image {i + 1}\n with Gaussian filter for Gaussian noise')
    axs[0].legend()

    axs[1].set_xlabel('Kernel size')
    axs[1].set_ylabel('MSE')
    axs[1].set_title(f'MSE for image {i + 1}\n with Gaussian filter for Salt_Pepper noise')
    axs[1].legend()



#### PSNR

In [ ]:
#images_with_gaussian_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}



In [ ]:
# plot each image on same plot plot for each gaus value
for i in range(len(images)):
    # print_with_border(f"Showing MSE for image {i + 1}:")
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for j in range(len(gaus_values)):
        axs[0].plot(kernel_sizes, PSNR_images_with_gaussian_filter_gaussian[i][j], label=f'Gaussian noise: {gaus_values[j]}')
        
    for k in range(len(salt_pepper_values)):
        axs[1].plot(kernel_sizes, PSNR_images_with_gaussian_filter_salt_pepper[i][k], label=f'Salt_Pepper noise: {salt_pepper_values[k]}')

    axs[0].set_xlabel('Kernel size')
    axs[0].set_ylabel('PSNR')
    axs[0].set_title(f'PSNR for image {i + 1}\n with Gaussian filter for Gaussian noise')
    axs[0].legend()




    axs[1].set_xlabel('Kernel size')
    axs[1].set_ylabel('PSNR')
    axs[1].set_title(f'PSNR for image {i + 1}\n with Gaussian filter for Salt_Pepper noise')
    axs[1].legend()



#### Edge preservation

In [ ]:
original_edges = [cv2.Canny(img, 100, 200) for img in images]

##### Gaussian Noise

In [ ]:

EDGES_images_with_gaussian_filter_gaussian= {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in range(len(kernel_sizes)) :
            # print(i, j, k)
            EDGES_images_with_gaussian_filter_gaussian[i][j].append(cv2.Canny(images_with_gaussian_filter_gaussian[i][j][k], 100, 200))
                                    

In [ ]:

for i in range(len(images)):
    print_with_border(f"Showing Edges for image {i + 1} with Gaussian filter for Gaussian noise:")

    for j in range(len(gaus_values)):
        print(f"\n\tShowing Edges for Gaussian noise {gaus_values[j]}:")

        fig, axs = plt.subplots(1, len(kernel_sizes)+1, figsize=(10, 10))

        axs[0].imshow(original_edges[i], cmap='gray')
        axs[0].axis('off')
        axs[0].set_title('Original Edges')

        for k in range(len(kernel_sizes)):
            axs[k+1].imshow(EDGES_images_with_gaussian_filter_gaussian[i][j][k], cmap='gray')
            axs[k+1].axis('off')
            axs[k+1].set_title(f'Edges: Gaussian filter: {kernel_sizes[k]}')

        plt.tight_layout()

        plt.show()

##### Salt_Pepper Noise

In [ ]:

EDGES_images_with_gaussian_filter_salt_pepper= {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in range(len(kernel_sizes)) :
            # print(i, j, k)
            EDGES_images_with_gaussian_filter_salt_pepper[i][j].append(cv2.Canny(images_with_gaussian_filter_salt_pepper[i][j][k], 100, 200))
                                                

In [ ]:

for i in range(len(images)):
    print_with_border(f"Showing Edges for image {i + 1} with Gaussian filter for Salt_Pepper noise:")

    for j in range(len(salt_pepper_values)):
        print(f"\n\tShowing Edges for Salt_Pepper noise {salt_pepper_values[j]}:")

        fig, axs = plt.subplots(1, len(kernel_sizes)+1, figsize=(10, 10))

        axs[0].imshow(original_edges[i], cmap='gray')
        axs[0].axis('off')
        axs[0].set_title('Original Edges')

        for k in range(len(kernel_sizes)):
            axs[k+1].imshow(EDGES_images_with_gaussian_filter_salt_pepper[i][j][k], cmap='gray')
            axs[k+1].axis('off')
            axs[k+1].set_title(f'Edges: Gaussian filter: {kernel_sizes[k]}')

        plt.tight_layout()

        plt.show()

### Median Filter

#### MSE

In [ ]:
#images_with_gaussian_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}




In [ ]:
# plot each image on same plot plot for each gaus value
for i in range(len(images)):
    # print_with_border(f"Showing MSE for image {i + 1}:")
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for j in range(len(gaus_values)):
        axs[0].plot(kernel_sizes, MSE_images_with_median_filter_gaussian[i][j], label=f'Gaussian noise: {gaus_values[j]}')
        
    for k in range(len(salt_pepper_values)):
        axs[1].plot(kernel_sizes, MSE_images_with_median_filter_salt_pepper[i][k], label=f'Salt_Pepper noise: {salt_pepper_values[k]}')

    axs[0].set_xlabel('Kernel size')
    axs[0].set_ylabel('MSE')
    axs[0].set_title(f'MSE for image {i + 1}\n with Median filter for Gaussian noise')
    axs[0].legend()




    axs[1].set_xlabel('Kernel size')
    axs[1].set_ylabel('MSE')
    axs[1].set_title(f'MSE for image {i + 1}\n with Median filter for Salt_Pepper noise')
    axs[1].legend()



#### PSNR

In [ ]:
#images_with_gaussian_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}




In [ ]:
# plot each image on same plot plot for each gaus value
for i in range(len(images)):
    # print_with_border(f"Showing MSE for image {i + 1}:")
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for j in range(len(gaus_values)):
        axs[0].plot(kernel_sizes, PSNR_images_with_median_filter_gaussian[i][j], label=f'Gaussian noise: {gaus_values[j]}')
        # axs[1].plot(kernel_sizes, PSNR_images_with_median_filter_salt_pepper[i][j], label=f'Salt_Pepper noise: {salt_pepper_values[j]}')

    for k in range(len(salt_pepper_values)):
        axs[1].plot(kernel_sizes, PSNR_images_with_median_filter_salt_pepper[i][k], label=f'Salt_Pepper noise: {salt_pepper_values[k]}')

        
    axs[0].set_xlabel('Kernel size')
    axs[0].set_ylabel('PSNR')
    axs[0].set_title(f'PSNR for image {i + 1}\n with Median filter for Gaussian noise')
    axs[0].legend()




    axs[1].set_xlabel('Kernel size')
    axs[1].set_ylabel('PSNR')
    axs[1].set_title(f'PSNR for image {i + 1}\n with Median filter for Salt_Pepper noise')
    axs[1].legend()



#### Edge preservation

In [ ]:
original_edges = [cv2.Canny(img, 100, 200) for img in images]

##### Gaussian Noise

In [ ]:

EDGES_images_with_median_filter_gaussian= {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in range(len(kernel_sizes)) :
            # print(i, j, k)
            EDGES_images_with_median_filter_gaussian[i][j].append(cv2.Canny(images_with_median_filter_gaussian[i][j][k], 100, 200))

In [ ]:

for i in range(len(images)):
    print_with_border(f"Showing Edges for image {i + 1} with Median filter for Gaussian noise:")

    for j in range(len(gaus_values)):
        print(f"\n\tShowing Edges for Gaussian noise {gaus_values[j]}:")

        fig, axs = plt.subplots(1, len(kernel_sizes)+1, figsize=(10, 10))

        axs[0].imshow(original_edges[i], cmap='gray')
        axs[0].axis('off')
        axs[0].set_title('Original Edges')

        for k in range(len(kernel_sizes)):
            axs[k+1].imshow(EDGES_images_with_median_filter_gaussian[i][j][k], cmap='gray')
            axs[k+1].axis('off')
            axs[k+1].set_title(f'Edges: Median filter: {kernel_sizes[k]}')

        plt.tight_layout()

        plt.show()

##### Salt_Pepper Noise

In [ ]:

EDGES_images_with_median_filter_salt_pepper= {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in range(len(kernel_sizes)) :
            # print(i, j, k)
            EDGES_images_with_median_filter_salt_pepper[i][j].append(cv2.Canny(images_with_median_filter_salt_pepper[i][j][k], 100, 200))

            

In [ ]:

for i in range(len(images)):
    print_with_border(f"Showing Edges for image {i + 1} with Median filter for Salt_Pepper noise:")

    for j in range(len(salt_pepper_values)):
        print(f"\n\tShowing Edges for Salt_Pepper noise {salt_pepper_values[j]}:")

        fig, axs = plt.subplots(1, len(kernel_sizes)+1, figsize=(10, 10))

        axs[0].imshow(original_edges[i], cmap='gray')
        axs[0].axis('off')
        axs[0].set_title('Original Edges')

        for k in range(len(kernel_sizes)):
            axs[k+1].imshow(EDGES_images_with_median_filter_salt_pepper[i][j][k], cmap='gray')
            axs[k+1].axis('off')
            axs[k+1].set_title(f'Edges: Median filter: {kernel_sizes[k]}')

        plt.tight_layout()

        plt.show()

## Advanced filters

### Adaptive Mean Filter

#### MSE

In [ ]:

#images_with_gaussian_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}



In [ ]:

# plot each image on same plot plot for each gaus value
for i in range(len(images)):
    # print_with_border(f"Showing MSE for image {i + 1}:")
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for j in range(len(gaus_values)):
        axs[0].plot(kernel_sizes, MSE_images_with_AdaptiveMean_filter_gaussian[i][j], label=f'Gaussian noise: {gaus_values[j]}')
        
    for k in range(len(salt_pepper_values)):
        axs[1].plot(kernel_sizes, MSE_images_with_AdaptiveMean_filter_salt_pepper[i][k], label=f'Salt_Pepper noise: {salt_pepper_values[k]}')

    axs[0].set_xlabel('Kernel size')
    axs[0].set_ylabel('MSE')
    axs[0].set_title(f'MSE for image {i + 1}\n with Adaptive Mean filter for Gaussian noise')
    axs[0].legend()




    axs[1].set_xlabel('Kernel size')
    axs[1].set_ylabel('MSE')
    axs[1].set_title(f'MSE for image {i + 1}\n with Adaptive Mean filter for Salt_Pepper noise')
    axs[1].legend()

#### PSNR

In [ ]:

PSNR_images_with_AdaptiveMean_filter_gaussian= calculate_PSNR_for_filters(images, images_with_AdaptiveMean_filter_gaussian, kernel_sizes)
PSNR_images_with_AdaptiveMean_filter_salt_pepper= calculate_PSNR_for_filters(images, images_with_AdaptiveMean_filter_salt_pepper, kernel_sizes)

In [ ]:

# plot each image on same plot plot for each gaus value

for i in range(len(images)):
    # print_with_border(f"Showing MSE for image {i + 1}:")
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for j in range(len(gaus_values)):
        axs[0].plot(kernel_sizes, PSNR_images_with_AdaptiveMean_filter_gaussian[i][j], label=f'Gaussian noise: {gaus_values[j]}')
        
    for k in range(len(salt_pepper_values)):
        axs[1].plot(kernel_sizes, PSNR_images_with_AdaptiveMean_filter_salt_pepper[i][k], label=f'Salt_Pepper noise: {salt_pepper_values[k]}')

    axs[0].set_xlabel('Kernel size')
    axs[0].set_ylabel('PSNR')
    axs[0].set_title(f'PSNR for image {i + 1}\n with Adaptive Mean filter for Gaussian noise')
    axs[0].legend()




    axs[1].set_xlabel('Kernel size')
    axs[1].set_ylabel('PSNR')
    axs[1].set_title(f'PSNR for image {i + 1}\n with Adaptive Mean filter for Salt_Pepper noise')
    axs[1].legend()

##### Salt_Pepper Noise

In [ ]:

EDGES_images_with_AdaptiveMean_filter_salt_pepper= {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in range(len(kernel_sizes)) :
            # print(i, j, k)
            EDGES_images_with_AdaptiveMean_filter_salt_pepper[i][j].append(cv2.Canny(images_with_AdaptiveMean_filter_salt_pepper[i][j][k], 100, 200))

In [ ]:

for i in range(len(images)):
    print_with_border(f"Showing Edges for image {i + 1} with Adaptive Mean filter for Salt_Pepper noise:")

    for j in range(len(salt_pepper_values)):
        print(f"\n\tShowing Edges for Salt_Pepper noise {salt_pepper_values[j]}:")

        fig, axs = plt.subplots(1, len(kernel_sizes)+1, figsize=(10, 10))

        axs[0].imshow(original_edges[i], cmap='gray')
        axs[0].axis('off')
        axs[0].set_title('Original Edges')

        for k in range(len(kernel_sizes)):
            axs[k+1].imshow(EDGES_images_with_AdaptiveMean_filter_salt_pepper[i][j][k], cmap='gray')
            axs[k+1].axis('off')
            axs[k+1].set_title(f'Edges: Adapt. Mean filter: {kernel_sizes[k]}')

        plt.tight_layout()

        plt.show()

### Bilateral Filter

#### MSE

In [ ]:

#images_with_gaussian_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

# Bilateral Filter


In [ ]:

# plot each image on same plot plot for each gaus value
for i in range(len(images)):
    # print_with_border(f"Showing MSE for image {i + 1}:")
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for j in range(len(gaus_values)):
        axs[0].plot(kernel_sizes, MSE_images_with_Bilateral_filter_gaussian[i][j], label=f'Gaussian noise: {gaus_values[j]}')
        
    for k in range(len(salt_pepper_values)):
        axs[1].plot(kernel_sizes, MSE_images_with_Bilateral_filter_salt_pepper[i][k], label=f'Salt_Pepper noise: {salt_pepper_values[k]}')

    axs[0].set_xlabel('Kernel size')
    axs[0].set_ylabel('MSE')
    axs[0].set_title(f'MSE for image {i + 1}\n with Bilateral filter for Gaussian noise')
    axs[0].legend()




    axs[1].set_xlabel('Kernel size')
    axs[1].set_ylabel('MSE')
    axs[1].set_title(f'MSE for image {i + 1}\n with Bilateral filter for Salt_Pepper noise')
    axs[1].legend()

    

#### PSNR

In [ ]:

# plot each image on same plot plot for each gaus value

for i in range(len(images)):
    # print_with_border(f"Showing MSE for image {i + 1}:")
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for j in range(len(gaus_values)):
        axs[0].plot(kernel_sizes, PSNR_images_with_Bilateral_filter_gaussian[i][j], label=f'Gaussian noise: {gaus_values[j]}')
        
    for k in range(len(salt_pepper_values)):
        axs[1].plot(kernel_sizes, PSNR_images_with_Bilateral_filter_salt_pepper[i][k], label=f'Salt_Pepper noise: {salt_pepper_values[k]}')

    axs[0].set_xlabel('Kernel size')
    axs[0].set_ylabel('PSNR')
    axs[0].set_title(f'PSNR for image {i + 1}\n with Bilateral filter for Gaussian noise')
    axs[0].legend()




    axs[1].set_xlabel('Kernel size')
    axs[1].set_ylabel('PSNR')
    axs[1].set_title(f'PSNR for image {i + 1}\n with Bilateral filter for Salt_Pepper noise')
    axs[1].legend()

##### Gaussian Noise

In [ ]:


EDGES_images_with_Bilateral_filter_gaussian= {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in range(len(kernel_sizes)) :
            # print(i, j, k)
            EDGES_images_with_Bilateral_filter_gaussian[i][j].append(cv2.Canny(images_with_Bilateral_filter_gaussian[i][j][k], 100, 200))

In [ ]:

for i in range(len(images)):
    print_with_border(f"Showing Edges for image {i + 1} with Bilateral filter for Gaussian noise:")

    for j in range(len(gaus_values)):
        print(f"\n\tShowing Edges for Gaussian noise {gaus_values[j]}:")

        fig, axs = plt.subplots(1, len(kernel_sizes)+1, figsize=(10, 10))

        axs[0].imshow(original_edges[i], cmap='gray')
        axs[0].axis('off')
        axs[0].set_title('Original Edges')

        for k in range(len(kernel_sizes)):
            axs[k+1].imshow(EDGES_images_with_Bilateral_filter_gaussian[i][j][k], cmap='gray')
            axs[k+1].axis('off')
            axs[k+1].set_title(f'Edges: Bilateral filter: {kernel_sizes[k]}')

        plt.tight_layout()

        plt.show()

##### Salt_Pepper Noise

In [ ]:

EDGES_images_with_Bilateral_filter_salt_pepper= {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}
for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in range(len(kernel_sizes)) :
            # print(i, j, k)
            EDGES_images_with_Bilateral_filter_salt_pepper[i][j].append(cv2.Canny(images_with_Bilateral_filter_salt_pepper[i][j][k], 100, 200))

In [ ]:

for i in range(len(images)):
    print_with_border(f"Showing Edges for image {i + 1} with Bilateral filter for Salt_Pepper noise:")

    for j in range(len(salt_pepper_values)):
        print(f"\n\tShowing Edges for Salt_Pepper noise {salt_pepper_values[j]}:")

        fig, axs = plt.subplots(1, len(kernel_sizes)+1, figsize=(10, 10))

        axs[0].imshow(original_edges[i], cmap='gray')
        axs[0].axis('off')
        axs[0].set_title('Original Edges')

        for k in range(len(kernel_sizes)):
            axs[k+1].imshow(EDGES_images_with_Bilateral_filter_salt_pepper[i][j][k], cmap='gray')
            axs[k+1].axis('off')
            axs[k+1].set_title(f'Edges: Bilateral filter: {kernel_sizes[k]}')

        plt.tight_layout()

        plt.show()

### Adaptive Median filter

#### MSE

In [ ]:

#images_with_gaussian_filter_gaussian = {0:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, 1:{noise1:[k3, k5, k7], noise2:[k3, k5, k7]}, ...}

#AdaptiveMedian Filter


In [ ]:

# plot each image on same plot plot for each gaus value

for i in range(len(images)):
    # print_with_border(f"Showing MSE for image {i + 1}:")
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for j in range(len(gaus_values)):
        axs[0].plot(kernel_sizes, MSE_images_with_AdaptiveMedian_filter_gaussian[i][j], label=f'Gaussian noise: {gaus_values[j]}')
        
    for k in range(len(salt_pepper_values)):
        axs[1].plot(kernel_sizes, MSE_images_with_AdaptiveMedian_filter_salt_pepper[i][k], label=f'Salt_Pepper noise: {salt_pepper_values[k]}')

    axs[0].set_xlabel('Kernel size')
    axs[0].set_ylabel('MSE')
    axs[0].set_title(f'MSE for image {i + 1}\n with Adaptive Median filter for Gaussian noise')
    axs[0].legend()




    axs[1].set_xlabel('Kernel size')
    axs[1].set_ylabel('MSE')
    axs[1].set_title(f'MSE for image {i + 1}\n with Adaptive Median filter for Salt_Pepper noise')
    axs[1].legend()

#### PSNR

In [ ]:

for i in range(len(images)):
    # print_with_border(f"Showing MSE for image {i + 1}:")
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    for j in range(len(gaus_values)):
        axs[0].plot(kernel_sizes, PSNR_images_with_AdaptiveMedian_filter_gaussian[i][j], label=f'Gaussian noise: {gaus_values[j]}')
        
    for k in range(len(salt_pepper_values)):
        axs[1].plot(kernel_sizes, PSNR_images_with_AdaptiveMedian_filter_salt_pepper[i][k], label=f'Salt_Pepper noise: {salt_pepper_values[k]}')

    axs[0].set_xlabel('Kernel size')
    axs[0].set_ylabel('PSNR')
    axs[0].set_title(f'PSNR for image {i + 1}\n with Adaptive Median filter for Gaussian noise')
    axs[0].legend()




    axs[1].set_xlabel('Kernel size')
    axs[1].set_ylabel('PSNR')
    axs[1].set_title(f'PSNR for image {i + 1}\n with Adaptive Median filter for Salt_Pepper noise')
    axs[1].legend()

##### Gaussian Noise

In [ ]:

EDGES_images_with_AdaptiveMedian_filter_gaussian= {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}
for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in range(len(kernel_sizes)) :
            # print(i, j, k)
            EDGES_images_with_AdaptiveMedian_filter_gaussian[i][j].append(cv2.Canny(images_with_AdaptiveMedian_filter_gaussian[i][j][k], 100, 200))

In [ ]:

for i in range(len(images)):
    print_with_border(f"Showing Edges for image {i + 1} with Adaptive Median filter for Gaussian noise:")

    for j in range(len(gaus_values)):
        print(f"\n\tShowing Edges for Gaussian noise {gaus_values[j]}:")

        fig, axs = plt.subplots(1, len(kernel_sizes)+1, figsize=(10, 10))

        axs[0].imshow(original_edges[i], cmap='gray')
        axs[0].axis('off')
        axs[0].set_title('Original Edges')

        for k in range(len(kernel_sizes)):
            axs[k+1].imshow(EDGES_images_with_AdaptiveMedian_filter_gaussian[i][j][k], cmap='gray')
            axs[k+1].axis('off')
            axs[k+1].set_title(f'Edges: Adapt. Median: {kernel_sizes[k]}')

        plt.tight_layout()

        plt.show()

##### Salt_Pepper Noise

In [ ]:

EDGES_images_with_AdaptiveMedian_filter_salt_pepper= {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in range(len(kernel_sizes)) :
            # print(i, j, k)
            EDGES_images_with_AdaptiveMedian_filter_salt_pepper[i][j].append(cv2.Canny(images_with_AdaptiveMedian_filter_salt_pepper[i][j][k], 100, 200))
                                                      

In [ ]:

for i in range(len(images)):
    print_with_border(f"Showing Edges for image {i + 1} with Adaptive Median filter for Salt_Pepper noise:")

    for j in range(len(salt_pepper_values)):
        print(f"\n\tShowing Edges for Salt_Pepper noise {salt_pepper_values[j]}:")

        fig, axs = plt.subplots(1, len(kernel_sizes)+1, figsize=(10, 10))

        axs[0].imshow(original_edges[i], cmap='gray')
        axs[0].axis('off')
        axs[0].set_title('Original Edges')

        for k in range(len(kernel_sizes)):
            axs[k+1].imshow(EDGES_images_with_AdaptiveMedian_filter_salt_pepper[i][j][k], cmap='gray')
            axs[k+1].axis('off')
            axs[k+1].set_title(f'Edges: Adapt. Median: {kernel_sizes[k]}')

        plt.tight_layout()

        plt.show()

# Time Results

In [ ]:


box_filter_average_time = calculate_average_time(time_images_with_box_filter_gaussian, time_images_with_box_filter_salt_pepper,images, kernel_sizes, gaus_values, salt_pepper_values)
gaussian_filter_average_time = calculate_average_time(time_images_with_gaussian_filter_gaussian, time_images_with_gaussian_filter_salt_pepper,images, kernel_sizes, gaus_values, salt_pepper_values)
median_filter_average_time = calculate_average_time(time_images_with_median_filter_gaussian, time_images_with_median_filter_salt_pepper,images, kernel_sizes, gaus_values, salt_pepper_values)
AdaptiveMean_filter_average_time = calculate_average_time(time_images_with_AdaptiveMean_filter_gaussian, time_images_with_AdaptiveMean_filter_salt_pepper,images, kernel_sizes, gaus_values, salt_pepper_values)
Bilateral_filter_average_time = calculate_average_time(time_images_with_Bilateral_filter_gaussian, time_images_with_Bilateral_filter_salt_pepper,images, kernel_sizes, gaus_values, salt_pepper_values)
AdaptiveMedian_filter_average_time = calculate_average_time(time_images_with_AdaptiveMedian_filter_gaussian, time_images_with_AdaptiveMedian_filter_salt_pepper,images, kernel_sizes, gaus_values, salt_pepper_values)

print(box_filter_average_time)
print(gaussian_filter_average_time)
print(median_filter_average_time)
print(AdaptiveMean_filter_average_time)
print(Bilateral_filter_average_time)
print(AdaptiveMedian_filter_average_time)


In [ ]:
# make all on one table

time_df = pd.DataFrame({ 'Box filter': box_filter_average_time,
                         'Gaussian filter': gaussian_filter_average_time,
                           'Median filter': median_filter_average_time,
                             'Adaptive Mean filter': AdaptiveMean_filter_average_time,
                               'Bilateral filter': Bilateral_filter_average_time,
                                 'Adaptive Median filter': AdaptiveMedian_filter_average_time})

In [ ]:
time_df

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10, 5))
width = 0.4
for i, col in enumerate(time_df.columns):
    axs.bar([k + i*width for k in time_df.index], time_df[col], width=width, label=col)

axs.set_yscale('log')
axs.set_xticks(kernel_sizes)
axs.set_xlabel('Kernel size')
axs.set_ylabel('Average time taken (ms)')
axs.set_title('Average time taken for each filter')
axs.legend()
plt.grid('y')

plt.tight_layout()
plt.show()


#

In [ ]:
# Dictionaries for Gaussian noise
#{k3: [avg_noise1, avg_noise2,.....], k5: [avg_noise1, avg_noise2,.....], k7: [avg_noise1, avg_noise2,.....]}
MSE_images_with_box_filter_gaussian = {i: {n: 0 for n in range (len(gaus_values))} for i in range(len(kernel_sizes))}
MSE_images_with_gaussian_filter_gaussian = {i: {n: 0 for n in range (len(gaus_values))} for i in range(len(kernel_sizes))}
MSE_images_with_median_filter_gaussian = {i: {n: 0 for n in range (len(gaus_values))} for i in range(len(kernel_sizes))}
MSE_images_with_AdaptiveMean_filter_gaussian = {i: {n: 0 for n in range (len(gaus_values))} for i in range(len(kernel_sizes))}
MSE_images_with_Bilateral_filter_gaussian = {i: {n: 0 for n in range (len(gaus_values))} for i in range(len(kernel_sizes))}
MSE_images_with_AdaptiveMedian_filter_gaussian = {i: {n: 0 for n in range (len(gaus_values))} for i in range(len(kernel_sizes))}
                                                  

for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in range(len(kernel_sizes)) :
            # print(i, j, k)
            MSE_images_with_box_filter_gaussian[k][j]+=(MSE(images[i], images_with_box_filter_gaussian[i][j][k]))
            MSE_images_with_gaussian_filter_gaussian[k][j]+=(MSE(images[i], images_with_gaussian_filter_gaussian[i][j][k]))
            MSE_images_with_median_filter_gaussian[k][j]+=(MSE(images[i], images_with_median_filter_gaussian[i][j][k]))
            MSE_images_with_AdaptiveMean_filter_gaussian[k][j]+=(MSE(images[i], images_with_AdaptiveMean_filter_gaussian[i][j][k]))
            MSE_images_with_Bilateral_filter_gaussian[k][j]+=(MSE(images[i], images_with_Bilateral_filter_gaussian[i][j][k]))
            MSE_images_with_AdaptiveMedian_filter_gaussian[k][j]+=(MSE(images[i], images_with_AdaptiveMedian_filter_gaussian[i][j][k]))

            # MSE_images_with_gaussian_filter_gaussian[k][j].append(MSE(images[i], images_with_gaussian_filter_gaussian[i][j][k]))
            # MSE_images_with_median_filter_gaussian[k][j].append(MSE(images[i], images_with_median_filter_gaussian[i][j][k]))
            # MSE_images_with_AdaptiveMean_filter_gaussian[k][j].append(MSE(images[i], images_with_AdaptiveMean_filter_gaussian[i][j][k]))
            # MSE_images_with_Bilateral_filter_gaussian[k][j].append(MSE(images[i], images_with_Bilateral_filter_gaussian[i][j][k]))
            # MSE_images_with_AdaptiveMedian_filter_gaussian[k][j].append(MSE(images[i], images_with_AdaptiveMedian_filter_gaussian[i][j][k]))
            
MSE_images_with_box_filter_gaussian
# box_filter_average_MSE_gaussian = {k: 0 for k in kernel_sizes}
# gaussian_filter_average_MSE_gaussian = {k: 0 for k in kernel_sizes}
# median_filter_average_MSE_gaussian = {k: 0 for k in kernel_sizes}
# AdaptiveMean_filter_average_MSE_gaussian = {k: 0 for k in kernel_sizes}
# Bilateral_filter_average_MSE_gaussian = {k: 0 for k in kernel_sizes}
# AdaptiveMedian_filter_average_MSE_gaussian = {k: 0 for k in kernel_sizes}

# # Dictionaries for Salt-Pepper noise
# box_filter_average_MSE_salt_pepper = {k: 0 for k in kernel_sizes}
# gaussian_filter_average_MSE_salt_pepper = {k: 0 for k in kernel_sizes}
# median_filter_average_MSE_salt_pepper = {k: 0 for k in kernel_sizes}
# AdaptiveMean_filter_average_MSE_salt_pepper = {k: 0 for k in kernel_sizes}
# Bilateral_filter_average_MSE_salt_pepper = {k: 0 for k in kernel_sizes}
# AdaptiveMedian_filter_average_MSE_salt_pepper = {k: 0 for k in kernel_sizes}

# # Calculate MSE for Gaussian noise
# for i in range(len(images)):
#     for k in range(len(kernel_sizes)):
#         for j in range(len(gaus_values)):
#             box_filter_average_MSE_gaussian[kernel_sizes[k]] += MSE_images_with_box_filter_gaussian[i][j][k]
#             gaussian_filter_average_MSE_gaussian[kernel_sizes[k]] += MSE_images_with_gaussian_filter_gaussian[i][j][k]
#             median_filter_average_MSE_gaussian[kernel_sizes[k]] += MSE_images_with_median_filter_gaussian[i][j][k]
#             AdaptiveMean_filter_average_MSE_gaussian[kernel_sizes[k]] += MSE_images_with_AdaptiveMean_filter_gaussian[i][j][k]
#             Bilateral_filter_average_MSE_gaussian[kernel_sizes[k]] += MSE_images_with_Bilateral_filter_gaussian[i][j][k]
#             AdaptiveMedian_filter_average_MSE_gaussian[kernel_sizes[k]] += MSE_images_with_AdaptiveMedian_filter_gaussian[i][j][k]

# for k in kernel_sizes:
#     box_filter_average_MSE_gaussian[k] /= (len(images) * len(gaus_values))
#     gaussian_filter_average_MSE_gaussian[k] /= (len(images) * len(gaus_values))
#     median_filter_average_MSE_gaussian[k] /= (len(images) * len(gaus_values))
#     AdaptiveMean_filter_average_MSE_gaussian[k] /= (len(images) * len(gaus_values))
#     Bilateral_filter_average_MSE_gaussian[k] /= (len(images) * len(gaus_values))
#     AdaptiveMedian_filter_average_MSE_gaussian[k] /= (len(images) * len(gaus_values))

# # Calculate MSE for Salt-Pepper noise
# for i in range(len(images)):
#     for k in range(len(kernel_sizes)):
#         for j in range(len(salt_pepper_values)):
#             box_filter_average_MSE_salt_pepper[kernel_sizes[k]] += MSE_images_with_box_filter_salt_pepper[i][j][k]
#             gaussian_filter_average_MSE_salt_pepper[kernel_sizes[k]] += MSE_images_with_gaussian_filter_salt_pepper[i][j][k]
#             median_filter_average_MSE_salt_pepper[kernel_sizes[k]] += MSE_images_with_median_filter_salt_pepper[i][j][k]
#             AdaptiveMean_filter_average_MSE_salt_pepper[kernel_sizes[k]] += MSE_images_with_AdaptiveMean_filter_salt_pepper[i][j][k]
#             Bilateral_filter_average_MSE_salt_pepper[kernel_sizes[k]] += MSE_images_with_Bilateral_filter_salt_pepper[i][j][k]
#             AdaptiveMedian_filter_average_MSE_salt_pepper[kernel_sizes[k]] += MSE_images_with_AdaptiveMedian_filter_salt_pepper[i][j][k]

# for k in kernel_sizes:
#     box_filter_average_MSE_salt_pepper[k] /= (len(images) * len(salt_pepper_values))
#     gaussian_filter_average_MSE_salt_pepper[k] /= (len(images) * len(salt_pepper_values))
#     median_filter_average_MSE_salt_pepper[k] /= (len(images) * len(salt_pepper_values))
#     AdaptiveMean_filter_average_MSE_salt_pepper[k] /= (len(images) * len(salt_pepper_values))
#     Bilateral_filter_average_MSE_salt_pepper[k] /= (len(images) * len(salt_pepper_values))
#     AdaptiveMedian_filter_average_MSE_salt_pepper[k] /= (len(images) * len(salt_pepper_values))


In [ ]:

# # make all on one table

# # MSE_gaussian_df = pd.DataFrame({ 'Box filter': box_filter_average_MSE_gaussian,
# #                           'Gaussian filter': gaussian_filter_average_MSE_gaussian,
# #                             'Median filter': median_filter_average_MSE_gaussian,
# #                               'Adaptive Mean filter': AdaptiveMean_filter_average_MSE_gaussian,
# #                                 'Bilateral filter': Bilateral_filter_average_MSE_gaussian,
# #                                   'Adaptive Median filter': AdaptiveMedian_filter_average_MSE_gaussian})

# # MSE_salt_pepper_df = pd.DataFrame({ 'Box filter': box_filter_average_MSE_salt_pepper,
# #                           'Gaussian filter': gaussian_filter_average_MSE_salt_pepper,
# #                             'Median filter': median_filter_average_MSE_salt_pepper,
# #                               'Adaptive Mean filter': AdaptiveMean_filter_average_MSE_salt_pepper,
# #                                 'Bilateral filter': Bilateral_filter_average_MSE_salt_pepper,

# #                                   'Adaptive Median filter': AdaptiveMedian_filter_average_MSE_salt_pepper})


# # MSE_images_with_box_filter_gaussian tKE THE MEAN 


# MSE_images_with_box_filter_gaussian_df = pd.DataFrame(MSE_images_with_box_filter_gaussian)
# MSE_images_with_gaussian_filter_gaussian_df = pd.DataFrame(MSE_images_with_gaussian_filter_gaussian)
# MSE_images_with_median_filter_gaussian_df = pd.DataFrame(MSE_images_with_median_filter_gaussian)
# MSE_images_with_AdaptiveMean_filter_gaussian_df = pd.DataFrame(MSE_images_with_AdaptiveMean_filter_gaussian)
# MSE_images_with_Bilateral_filter_gaussian_df = pd.DataFrame(MSE_images_with_Bilateral_filter_gaussian)
# MSE_images_with_AdaptiveMedian_filter_gaussian_df = pd.DataFrame(MSE_images_with_AdaptiveMedian_filter_gaussian)
# #transpose the dataframes

# fig, axs = plt.subplots(nrows=3,ncols= 2, figsize=(10, 5))
# axs[0, 0].plot(kernel_sizes, MSE_images_with_box_filter_gaussian_df.T)
# axs[0, 0].set_title('Box filter')
# axs[0, 0].set_xlabel('Kernel size')
# axs[0, 0].set_ylabel('MSE')
# axs[0, 0].legend()

# axs[0, 1].plot(kernel_sizes, MSE_images_with_gaussian_filter_gaussian_df.T)
# axs[0, 1].set_title('Gaussian filter')
# axs[0, 1].set_xlabel('Kernel size')
# axs[0, 1].set_ylabel('MSE')

# axs[1, 0].plot(kernel_sizes, MSE_images_with_median_filter_gaussian_df.T)
# axs[1, 0].set_title('Median filter')
# axs[1, 0].set_xlabel('Kernel size')
# axs[1, 0].set_ylabel('MSE')

# axs[1, 1].plot(kernel_sizes, MSE_images_with_AdaptiveMean_filter_gaussian_df.T)
# axs[1, 1].set_title('Adaptive Mean filter')
# axs[1, 1].set_xlabel('Kernel size')
# axs[1, 1].set_ylabel('MSE')

# axs[2, 0].plot(kernel_sizes, MSE_images_with_Bilateral_filter_gaussian_df.T)
# axs[2, 0].set_title('Bilateral filter')
# axs[2, 0].set_xlabel('Kernel size')
# axs[2, 0].set_ylabel('MSE')

# axs[2, 1].plot(kernel_sizes, MSE_images_with_AdaptiveMedian_filter_gaussian_df.T)
# axs[2, 1].set_title('Adaptive Median filter')
# axs[2, 1].set_xlabel('Kernel size')
# axs[2, 1].set_ylabel('MSE')

# plt.tight_layout()
# plt.show()

In [ ]:

fig, axs = plt.subplots(1, 1, figsize=(10, 5))
width = 0.4
for i, col in enumerate(MSE_gaussian_df.columns):
    axs.bar([k + i*width for k in MSE_gaussian_df.index], MSE_gaussian_df[col], width=width, label=col)

# axs.set_yscale('log')
axs.set_xticks(kernel_sizes)
axs.set_xlabel('Kernel size')

axs.set_ylabel('Average MSE')
axs.set_title('Average MSE for each filter with Gaussian noise')
axs.legend()
axs.set_yticks(np.arange(0, 100, 10))
axs.grid('y')


plt.tight_layout()
plt.show()

fig, axs = plt.subplots(1, 1, figsize=(10, 5))
width = 0.4
for i, col in enumerate(MSE_salt_pepper_df.columns):
    axs.bar([k + i*width for k in MSE_salt_pepper_df.index], MSE_salt_pepper_df[col], width=width, label=col)

axs.set_xticks(kernel_sizes)
axs.set_xlabel('Kernel size')
axs.set_yticks(np.arange(0, 100, 10))
axs.set_ylabel('Average MSE')
axs.set_title('Average MSE for each filter with Salt-Pepper noise')
axs.legend()
axs.grid('y')  


# Edge Preservation

## Simple Filters

### Box Filter

#### Gaussian Noise

In [ ]:
EDGES_images_with_box_filter_gaussian= {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in range(len(kernel_sizes)) :
            # print(i, j, k)
            EDGES_images_with_box_filter_gaussian[i][j].append(cv2.Canny(images_with_box_filter_gaussian[i][j][k], 100, 200))



plot_with_edges(images=images,
                images_with_edges=EDGES_images_with_box_filter_gaussian,
                filter_type='Box',
                noise_type='gaussian',
                noise_values=gaus_values,
                kernel_sizes=kernel_sizes,
                kernel_sizes_to_show=kernel_sizes_to_show)

#### Salt_Pepper Noise

In [ ]:

EDGES_images_with_box_filter_salt_pepper= {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in range(len(kernel_sizes)) :
            EDGES_images_with_box_filter_salt_pepper[i][j].append(cv2.Canny(images_with_box_filter_salt_pepper[i][j][k], 100, 200))

plot_with_edges(images=images,
                images_with_edges=EDGES_images_with_box_filter_salt_pepper,
                filter_type='Box',
                noise_type='salt_pepper',
                noise_values=salt_pepper_values,
                kernel_sizes=kernel_sizes,
                kernel_sizes_to_show=kernel_sizes_to_show)


### Gaussian Filter

#### Gaussian Noise

In [ ]:
EDGES_images_with_gaussian_filter_gaussian= {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}
for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in range(len(kernel_sizes)) :
            EDGES_images_with_gaussian_filter_gaussian[i][j].append(cv2.Canny(images_with_gaussian_filter_gaussian[i][j][k], 100, 200))

plot_with_edges(images=images,
                images_with_edges=EDGES_images_with_gaussian_filter_gaussian,
                filter_type='Gaussian',
                noise_type='gaussian',
                noise_values=gaus_values,
                kernel_sizes=kernel_sizes,
                kernel_sizes_to_show=kernel_sizes_to_show)

#### Salt_Pepper Noise

In [ ]:

EDGES_images_with_gaussian_filter_salt_pepper= {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}
for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in range(len(kernel_sizes)) :
            EDGES_images_with_gaussian_filter_salt_pepper[i][j].append(cv2.Canny(images_with_gaussian_filter_salt_pepper[i][j][k], 100, 200))

plot_with_edges(images=images,
                images_with_edges=EDGES_images_with_gaussian_filter_salt_pepper,
                filter_type='Gaussian',
                noise_type='salt_pepper',
                noise_values=salt_pepper_values,
                kernel_sizes=kernel_sizes,
                kernel_sizes_to_show=kernel_sizes_to_show)

### Median Filter

#### Gaussian Noise

In [ ]:

EDGES_images_with_median_filter_gaussian= {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}
for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in range(len(kernel_sizes)) :
            EDGES_images_with_median_filter_gaussian[i][j].append(cv2.Canny(images_with_median_filter_gaussian[i][j][k], 100, 200))

plot_with_edges(images=images,
                images_with_edges=EDGES_images_with_median_filter_gaussian,
                filter_type='Median',
                noise_type='gaussian',
                noise_values=gaus_values,
                kernel_sizes=kernel_sizes,
                kernel_sizes_to_show=kernel_sizes_to_show)

#### Salt_Pepper Noise

In [ ]:
EDGES_images_with_median_filter_salt_pepper= {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}
for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in range(len(kernel_sizes)) :
            EDGES_images_with_median_filter_salt_pepper[i][j].append(cv2.Canny(images_with_median_filter_salt_pepper[i][j][k], 100, 200))

plot_with_edges(images=images,
                images_with_edges=EDGES_images_with_median_filter_salt_pepper,
                filter_type='Median',
                noise_type='salt_pepper',
                noise_values=salt_pepper_values,
                kernel_sizes=kernel_sizes,
                kernel_sizes_to_show=kernel_sizes_to_show)

## Advanced filters

### Adaptive Mean Filter

#### Gaussian Noise

In [ ]:
EDGES_images_with_AdaptiveMean_filter_gaussian= {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}
for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in range(len(kernel_sizes)) :
            EDGES_images_with_AdaptiveMean_filter_gaussian[i][j].append(cv2.Canny(images_with_AdaptiveMean_filter_gaussian[i][j][k], 100, 200))

plot_with_edges(images=images,
                images_with_edges=EDGES_images_with_AdaptiveMean_filter_gaussian,
                filter_type='Adaptive Mean',
                noise_type='gaussian',
                noise_values=gaus_values,
                kernel_sizes=kernel_sizes,
                kernel_sizes_to_show=kernel_sizes_to_show)

#### Salt_Pepper Noise

In [ ]:
EDGES_images_with_AdaptiveMean_filter_salt_pepper= {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in range(len(kernel_sizes)) :
            EDGES_images_with_AdaptiveMean_filter_salt_pepper[i][j].append(cv2.Canny(images_with_AdaptiveMean_filter_salt_pepper[i][j][k], 100, 200))

plot_with_edges(images=images,
                images_with_edges=EDGES_images_with_AdaptiveMean_filter_salt_pepper,
                filter_type='Adaptive Mean',
                noise_type='salt_pepper',
                noise_values=salt_pepper_values,
                kernel_sizes=kernel_sizes,
                kernel_sizes_to_show=kernel_sizes_to_show)

### Bilateral Filter

#### Gaussian Noise

In [ ]:
EDGES_images_with_Bilateral_filter_gaussian= {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}
for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in range(len(kernel_sizes)) :
            EDGES_images_with_Bilateral_filter_gaussian[i][j].append(cv2.Canny(images_with_Bilateral_filter_gaussian[i][j][k], 100, 200))

plot_with_edges(images=images,
                images_with_edges=EDGES_images_with_Bilateral_filter_gaussian,
                filter_type='Bilateral',
                noise_type='gaussian',
                noise_values=gaus_values,
                kernel_sizes=kernel_sizes,
                kernel_sizes_to_show=kernel_sizes_to_show)

#### Salt_Pepper Noise

In [ ]:
EDGES_images_with_Bilateral_filter_salt_pepper= {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}
for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in range(len(kernel_sizes)) :
            EDGES_images_with_Bilateral_filter_salt_pepper[i][j].append(cv2.Canny(images_with_Bilateral_filter_salt_pepper[i][j][k], 100, 200))

plot_with_edges(images=images,
                images_with_edges=EDGES_images_with_Bilateral_filter_salt_pepper,
                filter_type='Bilateral',
                noise_type='salt_pepper',
                noise_values=salt_pepper_values,
                kernel_sizes=kernel_sizes,
                kernel_sizes_to_show=kernel_sizes_to_show)

### Adaptive Mean Filter

#### Gaussian Noise

In [ ]:
EDGES_images_with_AdaptiveMedian_filter_gaussian= {i: {n: [] for n in range (len(gaus_values))} for i in range(len(images))}
for i in range(len(images)):
    for j in range(len(gaus_values)):
        for k in range(len(kernel_sizes)) :
            EDGES_images_with_AdaptiveMedian_filter_gaussian[i][j].append(cv2.Canny(images_with_AdaptiveMedian_filter_gaussian[i][j][k], 100, 200))

plot_with_edges(images=images,
                images_with_edges=EDGES_images_with_AdaptiveMedian_filter_gaussian,
                filter_type='Adaptive Median',
                noise_type='gaussian',
                noise_values=gaus_values,
                kernel_sizes=kernel_sizes,
                kernel_sizes_to_show=kernel_sizes_to_show)

#### Salt_Pepper Noise

In [ ]:
EDGES_images_with_AdaptiveMedian_filter_salt_pepper= {i: {n: [] for n in range (len(salt_pepper_values))} for i in range(len(images))}

for i in range(len(images)):
    for j in range(len(salt_pepper_values)):
        for k in range(len(kernel_sizes)) :
            EDGES_images_with_AdaptiveMedian_filter_salt_pepper[i][j].append(cv2.Canny(images_with_AdaptiveMedian_filter_salt_pepper[i][j][k], 100, 200))

plot_with_edges(images=images,
                images_with_edges=EDGES_images_with_AdaptiveMedian_filter_salt_pepper,
                filter_type='Adaptive Median',
                noise_type='salt_pepper',
                noise_values=salt_pepper_values,
                kernel_sizes=kernel_sizes,
                kernel_sizes_to_show=kernel_sizes_to_show)